In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from utils import get_rec_table_skinny, get_rec_table_skinny2, get_rec_table_wide, get_player_scatter_vertical, get_player_scatter_horizontal
# import nfl_data_py as nfl

import warnings
warnings.filterwarnings('ignore')

In [2]:
all_data = pd.read_csv(r"data\final_data.csv")

In [3]:
player_list = list(all_data.player_display_name.sort_values().unique())
# player_list

In [4]:
all_data.columns

Index(['player_display_name', 'position', 'team', 'opponent', 'book_stat',
       'pp_line', 'ud_line', 'player_id', 'player_name', 'season', 'week',
       'completions', 'attempts', 'passing_yards', 'passing_tds',
       'interceptions', 'sacks', 'carries', 'rushing_yards', 'rushing_tds',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_air_yards', 'target_share', 'wopr', 'fantasy_points',
       'opponent_team', 'pp_delta', 'ud_delta', 'team_long', 'spread',
       'over_under'],
      dtype='object')

In [6]:
###  SELECTBOX

player = 'Travis Kelce'  # from selectbox
all_data = all_data.astype({'season': int})

player_all = all_data[(all_data.player_display_name==player)].reset_index(drop=True)
player_season = player_all[player_all.season==2023].reset_index(drop=True)
player_season2 = player_all[player_all.season > 2021].reset_index(drop=True)

line = (player_all.pp_line.mean() + player_all.ud_line.mean())/2
title = f"{player_all.player_display_name[0]}"


In [14]:
get_rec_table_wide(player_season[player_season.book_stat=='receiving_yards']).style.hide(axis="index").format(precision=0)

week,receiving_yards,targets,receptions,receiving_tds
20,75,6,5,2
19,71,10,7,0
17,16,4,3,0
16,44,7,5,0
15,28,7,5,0
14,83,10,6,0
13,81,5,4,0
12,91,7,6,0
11,44,9,7,1
9,14,4,3,0


In [16]:



table = get_rec_table_wide(player_season[player_season.book_stat=='receiving_yards']).style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=0).hide(axis="index")
table = table.set_properties(**{'text-align': 'center'})




table

week,receiving_yards,targets,receptions,receiving_tds
20,75,6,5,2
19,71,10,7,0
17,16,4,3,0
16,44,7,5,0
15,28,7,5,0
14,83,10,6,0
13,81,5,4,0
12,91,7,6,0
11,44,9,7,1
9,14,4,3,0


In [13]:
get_rec_table_skinny(player_season[player_season.book_stat=='receiving_yards']).style.hide(axis="index").format(precision=0)

week,receiving_yards
20,75
19,71
17,16
16,44
15,28
14,83
13,81
12,91
11,44
9,14


In [ ]:
fig = get_player_scatter_vertical(player_season[player_season.book_stat=='receiving_yards'])
fig

In [ ]:
player_season[player_season.book_stat=='receiving_yards']

In [ ]:
# SCATTER PLOT  (POSSIBLE LIGHT OPTION)

fig = px.scatter(player_season[player_season.book_stat=='receiving_yards'],
          x='targets',
          y='receiving_yards',
          size='week',
          color='week',
          template='plotly_white',
          size_max=15,
          width=300,
          height=600,
          color_continuous_scale='blues',
          title = f"{player_all.player_display_name[0]}<br><b><span style='color:yellow'>{player_all.ud_line.mean()}</span><br><span style='color:purple'>{player_all.pp_line.mean()}</span><br>",
          labels={'receiving_yards':'Receiving Yards','targets':'Targets'})
fig.update_coloraxes(showscale=False)
fig.add_hline(y=player_all.ud_line.mean(), line_width=2, line_color="yellow")
fig.add_hline(y=player_all.pp_line.mean(), line_width=2, line_color="purple")
fig.update_layout(title_x=0.5)

In [ ]:
##   go.Table skinny

skinny = get_rec_table_skinny(player_season[player_season.book_stat=='receiving_yards'])

fig = go.Figure(data = [go.Table(header = dict(values = ['<br><b>Week</b>', '<b>Rec<br>Yards</b>'],
                                               align = "center"),
                                 cells = dict(values = [skinny.week, skinny.receiving_yards],
                                              align = "center",
                                               font = dict(color = "darkslategrey", size = 11, family = 'Courier New')))])
 
fig.update_layout(width=275, height=600)
table1 = fig
fig.show()

In [ ]:
##   go.Table wide

wide = get_rec_table_wide(player_season[player_season.book_stat=='receiving_yards'])

fig = go.Figure(data = [go.Table(columnwidth = [.7,1,.7,1,1],
                                 header = dict(values = ['<b>Week</b>','<b>Targets</b>','<b>Rec</b>','<b>Rec TDs</b>'],
                                               align = "center",
                                               font = dict(size=10, color='black')),
                                 cells = dict(values = [wide.week, wide.targets, wide.receptions, wide.receiving_tds],
                                              align = "center",
                                              font = dict(color = "darkslategrey", size = 13, family = 'Courier New')))])
 
fig.update_layout(width=550, height=600)
table2 = fig
fig.show()

In [ ]:
player_season2.head()

In [ ]:
## ONE PLAYER Receiving Yards Distribution (PLOTLY)

hist_fig = px.histogram(player_season2[player_season2.book_stat=='receiving_yards'],#[player_df.season > 2021].reset_index(drop=True),# & (nfl.season == 2023)],
            x='receiving_yards',
            barmode='stack',
            nbins=30,
            template='simple_white',
            text_auto='true',
            color='season',
            width = 700,
            height = 400,
            title=title
            )
hist_fig.show()

In [ ]:
###################################################################################
## ECDF Plot (Plotly) last 2 seasons

fig_ecdf = px.ecdf(player_season2[player_season2.book_stat=='receiving_yards'], y="receiving_yards", color="season",template='ggplot2',title=title,width=400)#,title=f"<b></b><br>Mean: {round(np.mean(chase.receiving_yards),1)} Median {np.median(chase.receiving_yards)}<br>Underdog Line: {chase.ud_line.mean()}<br><br>",width=600)#  Mean Projection: {round(chase.etr_proj.mean(),1)}<br><br>",width=600)
fig_ecdf.add_hline(y=line, line_width=3, line_color="yellow")
fig_ecdf.add_hline(y=player_season['receiving_yards'].mean(), line_width=2, line_color="black")
fig_ecdf.add_hline(y=player_season.receiving_yards.median(), line_width=2, line_color="red")
fig_ecdf.show()

In [ ]:
#####################################################################################
## Histogram w/ KDE line

### distplot with kde line
sns.histplot(player_season2[player_season2.book_stat=='receiving_yards'].receiving_yards,bins=20,kde=True,color='olive').set(title=title)
sns.set(rc={"figure.figsize":(5, 3)})
plt.axvline(np.median(player_season2.receiving_yards).round(0),linestyle='solid',color='black')
plt.axvline(line,linestyle='solid',color='yellow')

In [ ]:
## Histogram w/ KDE line (hue=season)

sns.histplot(player_season2[player_season2.book_stat=='receiving_yards'], x="receiving_yards", hue="season",bins=20, kde=True).set(title=title)
sns.set(rc={"figure.figsize":( 5, 3)})
plt.axvline(line,linestyle='solid',color='yellow')
plt.axvline(player_season2['receiving_yards'].median(),linestyle='solid',color='black')

In [ ]:
## ECDF Plot (matplotlib) last 2 seasons

sns.displot(data=player_season2[player_season2.book_stat=='receiving_yards'], x="receiving_yards", kind="ecdf", hue='season').set(title=title)
sns.set(rc={"figure.figsize":( 5, 3)})
plt.axvline(np.median(player_season.receiving_yards).round(0),linestyle='solid',color='black')
plt.axvline(line,linestyle='solid',color='yellow')
#     plt.show()

In [ ]:
player_season2[player_season2.book_stat=='receiving_yards']

In [ ]:
##  Horizontal scatter (showing last 2 seasons)

get_player_scatter_horizontal(player_season2[player_season2.book_stat=='receiving_yards'])

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    shared_xaxes=False,
    vertical_spacing=0.03,
    specs=[[{"type": "scatter"}, {"type": "table"}]],
)

temp = player_season[player_season.book_stat=='receiving_yards'][['week','receiving_yards','book_stat']]
# skinny_scatter = get_player_scatter_vertical(player_season)
# skinny_table = get_rec_table_skinny2(temp)
# HERE IS THE PX.SCATTER PLOT (commented out since i cannot add it)
for t in px.scatter(player_season,x='targets',y='receiving_yards',title='titel').data:
    fig.add_trace(t, row=1, col=1)

fig.add_trace(
    go.Table(
        header = dict(values = ['<br><b>Week</b>', '<b>Rec<br>Yards</b>'], align = "center"),
        cells = dict(
            values = [
                temp.week, 
                temp.receiving_yards,
            ],
            align = "center",
        ),
    ),
    row=1,
    col=2,
)
fig.show()


#     go.Table(
#         header=dict(values=['week','receiving_yards'], font=dict(size=10), align="left"),
#         cells=dict(
#             values=[
#                 temp.week,
#                 temp.receiving_yards,
#             ],
#             fill_color="lavender",
#             align="left",
#         ),
#     ),
#     row=1,
#     col=2,
# )

In [ ]:
get_multifig(player_season)